# Automating Restaurant Menu to Excel with AI

### This project converts restaurant menus from images or PDFs into structured Excel files, eliminating manual data entry. The free service helps restaurants quickly generate upload-ready spreadsheets, saving time and costs.

**Workflow:**  


` 
PDF → Image → Excel
`

In [43]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import base64
from IPython.display import display, Image, Markdown
import pandas as pd

In [44]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [45]:
# Set the OpenAI connection and model
MODEL = "gpt-4o"
client = OpenAI(api_key=api_key)

## Defining the System Prompt

### We define a detailed system prompt that instructs the GPT model on how to convert the menu images into a structured Excel format.

In [46]:
# Define the system prompt with detailed instructions
system_prompt = """
Convert the menu image to a structured excel sheet format following the provided template and instructions.
This assistant converts restaurant or cafe menu data into a structured Excel sheet that adheres to a specific template.
The template includes categories, subcategories, item names, prices, descriptions, and more, ensuring data consistency.
This assistant helps users fill out each row correctly, following the detailed instructions provided.

Overview:
- Each row in the Excel spreadsheet represents a unique item, categorized under a category or subcategory.
- Category and subcategory names are repeated for items within the same subcategory.
- Certain columns are left blank when not applicable, such as subcategory details for items directly under a category.
- Item details, including names, prices, and descriptions, must be unique for each entry.
- Uploaded menu content will be appended to the existing menu without deleting any current entries.

Columns Guide:

Column Name                    | Description                               | Accepted Values           | Example
-------------------------------|-------------------------------------------|---------------------------|-----------------------
CategoryTitlePt (Column A)      | Category names in Portuguese              | Text, 256 characters max  | Bebidas
CategoryTitleEn (Column B) (Optional) | English translations of category titles | Text, 256 characters max  | Beverages
SubcategoryTitlePt (Column C) (Optional) | Subcategory titles in Portuguese | Text, 256 characters max or blank | Sucos
SubcategoryTitleEn (Column D) (Optional) | English translations of subcategory titles | Text, 256 characters max or blank | Juices
ItemNamePt (Column E)           | Item names in Portuguese                  | Text, 256 characters max  | Água Mineral
ItemNameEn (Column F) (Optional) | English translations of item names | Text, 256 characters max or blank | Mineral Water
ItemPrice (Column G)          | Price of each item without currency symbol  | Text                      | 2.50 or 2,50
Calories (Column H) (Optional) | Caloric content of each item              | Numeric                   | 150
PortionSize (Column I)        | Portion size for each item in units        | Text                      | 500ml, 1, 2-3
Availability (Column J) (Optional) | Current availability of the item     | Numeric: 1 for Yes, 0 for No | 1
ItemDescriptionPt (Column K) (Optional) | Detailed description in Portuguese | Text, 500 characters max  | Contains essential minerals
ItemDescriptionEn (Column L) (Optional) | Detailed description in English | Text, 500 characters max  | Contains essential minerals

Notes:
- Ensure all data entered follows the specified formats to maintain database integrity.
- Review the data for accuracy and consistency before submitting the Excel sheet.
"""


In [57]:
directory =  f"{os.getcwd()}"
IMAGE_DIR =directory

The next cell defines a function to encode images in base64 format and lists all image files (PNG, JPG, JPEG) in the specified directory.


In [59]:
def encode_image(image_path):
    """
    Encodes an image to base64 format.
    """
    with open(image_path, "rb") as image_file:
        import base64
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

# Process images in the directory
try:
    image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
except FileNotFoundError:
    image_files = []
    print(f"Image directory not found: {IMAGE_DIR}")

image_files

['DimSum Amoreiras 1.PNG',
 'DimSum Amoreiras 2.PNG',
 'DimSum Amoreiras 3.PNG',
 'DimSum Amoreiras 4.PNG',
 'DimSum Amoreiras 5.PNG']

In [61]:
test = image_files[0]
test

'DimSum Amoreiras 1.PNG'

In [90]:
# Create the PANDAS dataframe
df = pd.DataFrame(columns=['CategoryTitlePt', 'CategoryTitleEn', 'SubcategoryTitlePt', 'SubcategoryTitleEn',
                           'ItemNamePt', 'ItemNameEn', 'ItemPrice', 'Calories', 'PortionSize', 'Availability',
                           'ItemDescriptionPt', 'ItemDescriptionEn'])

In [64]:
# Retrieve and encode the image
image_path = os.path.join(IMAGE_DIR, test)
image_data = encode_image(image_path)

# Use GPT-4o to analyze and convert the imae
response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": [
                {'type': 'text',
                'text': "Convert this menu image to a structured Excel Sheet Format."},
                {'type': 'image_url',
                'image_url': {'url': f'data:image/png;base64,{image_data}'}}
            ]}],
        temperature = 0
    )

In [91]:
from IPython.display import Markdown, display

display(Markdown(response.choices[0].message.content))

Here's the structured Excel sheet format based on the menu image:

| CategoryTitlePt | CategoryTitleEn | SubcategoryTitlePt | SubcategoryTitleEn | ItemNamePt                  | ItemNameEn                  | ItemPrice | Calories | PortionSize | Availability | ItemDescriptionPt                                      | ItemDescriptionEn                                      |
|-----------------|-----------------|--------------------|--------------------|-----------------------------|-----------------------------|-----------|----------|-------------|--------------|--------------------------------------------------------|--------------------------------------------------------|
| Sopas           | Soups           |                    |                    | Sopa Won Ton                | Won Ton Soup                | 3,95      |          | 1           | 1            | Paksoy, Porco e Camarão                                 | Paksoy, Pork & Shrimp                                  |
| Sopas           | Soups           |                    |                    | Sopa Vegetariana            | Vegetarian Soup             | 3,95      |          | 1           | 1            | Paksoy, Bambu, Cogumelo e Verduras                      | Paksoy, Bamboo, Mushroom & Vegetables                  |
| Arroz & Verduras| Rice & Vegetables|                    |                    | Arroz branco                | Steamed Rice                | 1,95      |          | 1           | 1            | Arroz ao vapor                                          | Steamed Rice                                           |
| Arroz & Verduras| Rice & Vegetables|                    |                    | Paksoy ao vapor             | Steamed Paksoy              | 2,40      |          | 1           | 1            | Paksoy ao vapor com Coentros                            | Steamed Paksoy with Coriander                          |
| Siao Long Pao   | Siao Long Pao   |                    |                    | Siao Long Pao Tradicional   | Traditional Siao Long Pao   | 4,50      |          | 3           | 1            | Porco, Paksoy e Cogumelo Shiitake                       | Pork, Paksoy & Shiitake Mushroom                       |
| Siao Long Pao   | Siao Long Pao   |                    |                    | Siao Long Pao Negro         | Black Siao Long Pao         | 4,50      |          | 3           | 1            | Porco, Gengibre e Cebolinho                             | Pork, Ginger & Chives                                  |
| Gyozas          | Gyozas          |                    |                    | Gyoza Vegetais              | Vegetable Gyoza             | 4,50      |          | 3           | 1            | Veg                                                    | Veg                                                    |
| Gyozas          | Gyozas          |                    |                    | Gyoza Camarão               | Shrimp Gyoza                | 4,50      |          | 3           | 1            | Shrimp                                                 | Shrimp                                                 |
| Gyozas          | Gyozas          |                    |                    | Gyoza Pato                  | Duck Gyoza                  | 4,50      |          | 3           | 1            | Duck                                                   | Duck                                                   |
| Gyozas          | Gyozas          |                    |                    | Gyoza Galinha               | Chicken Gyoza               | 4,50      |          | 3           | 1            | Chicken                                                | Chicken                                                |
| Pratos          | Dishes          |                    |                    | Frango BEIJING              | BEIJING Chicken             | 8,95      |          | 1           | 1            | Frango agridoce envolto em massa crocante               | Crispy Sweet & Sour Chicken                            |
| Pratos          | Dishes          |                    |                    | Camarão Penghu              | Penghu Shrimp               | 9,95      |          | 1           | 1            | Camarão agridoce envolto em massa crocante              | Crispy Sweet & Sour Shrimps                            |

This table captures the menu items, their descriptions, and prices in both Portuguese and English.

In [92]:
for row in response.choices[0].message.content.split('\n'):
    if row.startswith('|') and not row.startswith('|-'): # Ensure that the data is a row and not a header format
        columns = [col.strip() for col in row.split('|')[1:-1]]
        if len(columns) == len(df.columns): # We need to make sure it's following the same structure as the dataframe
            if 'CategoryTitlePt' in columns:
                continue
            new_row = pd.Series(columns, index=df.columns)
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True) # keep adding the rows to the dataframe
        else:
            print(f"Skipping row: {row}")

In [93]:
df.head()

,CategoryTitlePt,CategoryTitleEn,SubcategoryTitlePt,SubcategoryTitleEn,ItemNamePt,ItemNameEn,ItemPrice,Calories,PortionSize,Availability,ItemDescriptionPt,ItemDescriptionEn
0,Sopas,Soups,,,Sopa Won Ton,Won Ton Soup,"3,95",,1,1,"Paksoy, Porco e Camarão","Paksoy, Pork & Shrimp"
1,Sopas,Soups,,,Sopa Vegetariana,Vegetarian Soup,"3,95",,1,1,"Paksoy, Bambu, Cogumelo e Verduras","Paksoy, Bamboo, Mushroom & Vegetables"
2,Arroz & Verduras,Rice & Vegetables,,,Arroz branco,Steamed Rice,"1,95",,1,1,Arroz ao vapor,Steamed Rice
3,Arroz & Verduras,Rice & Vegetables,,,Paksoy ao vapor,Steamed Paksoy,"2,40",,1,1,Paksoy ao vapor com Coentros,Steamed Paksoy with Coriander
4,Siao Long Pao,Siao Long Pao,,,Siao Long Pao Tradicional,Traditional Siao Long Pao,"4,50",,3,1,"Porco, Paksoy e Cogumelo Shiitake","Pork, Paksoy & Shiitake Mushroom"


In [94]:
len(df)

12

### Add GenAI Extraction for all images

In [97]:
# Create the PANDAS dataframe
df = pd.DataFrame(columns=['CategoryTitlePt', 'CategoryTitleEn', 'SubcategoryTitlePt', 'SubcategoryTitleEn',
                           'ItemNamePt', 'ItemNameEn', 'ItemPrice', 'Calories', 'PortionSize', 'Availability',
                           'ItemDescriptionPt', 'ItemDescriptionEn'])

In [98]:
image_files

['DimSum Amoreiras 1.PNG',
 'DimSum Amoreiras 2.PNG',
 'DimSum Amoreiras 3.PNG',
 'DimSum Amoreiras 4.PNG',
 'DimSum Amoreiras 5.PNG']

## Process of extracting structured data from a set of menu images using a GenAI model (such as GPT-4o).
 
The workflow consists of the following phases:
1. **Image Retrieval and Encoding**: For each image file, the image is loaded and encoded in base64 format.
2. **GenAI Analysis**: The encoded image is sent to a GenAI model with a prompt to convert the menu image into a structured Excel sheet format.
3. **Data Extraction and Aggregation**: The model's response, formatted as a markdown table, is parsed row by row. Each valid row is added to a pandas DataFrame, building up a structured dataset from all images.

The overall process automates the conversion of unstructured menu images into a structured, tabular format suitable for further analysis or export.

Below is a flowchart illustrating the process:

```
+-----------------------------+
| 1. Image Retrieval & Encode |
+-------------+---------------+
             |
             v
+-----------------------------+
| 2. GenAI Model Analysis     |
|   (Image -> Table)          |
+-------------+---------------+
             |
             v
+-----------------------------+
| 3. Parse & Append Rows      |
|   (to DataFrame)            |
+-----------------------------+
```



In [99]:
for image in image_files:
    # Phase 1
    # Retrieve and encode the image
    image_path = os.path.join(IMAGE_DIR, image)
    image_data = encode_image(image_path)

    # Phase 2
    # Use GPT-4o to analyze and convert the imae
    response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": [
                    {'type': 'text',
                    'text': "Convert this menu image to a structured Excel Sheet Format."},
                    {'type': 'image_url',
                    'image_url': {'url': f'data:image/png;base64,{image_data}'}}
                ]}],
            temperature = 0
        )

    # Phase 3
    for row in response.choices[0].message.content.split('\n'):
        if row.startswith('|') and not row.startswith('|-'): # Ensure that the data is a row and not a header format
            columns = [col.strip() for col in row.split('|')[1:-1]]
            if len(columns) == len(df.columns): # We need to make sure it's following the same structure as the dataframe
                if 'CategoryTitlePt' in columns:
                    continue
                new_row = pd.Series(columns, index=df.columns)
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True) # keep adding the rows to the dataframe
            else:
                print(f"Skipping row: {row}")

In [101]:
df.head()

,CategoryTitlePt,CategoryTitleEn,SubcategoryTitlePt,SubcategoryTitleEn,ItemNamePt,ItemNameEn,ItemPrice,Calories,PortionSize,Availability,ItemDescriptionPt,ItemDescriptionEn
0,Sopas,Soups,,,Sopa Won Ton,Won Ton Soup,"3,95",,1,1,"Paksoy, Porco e Camarão","Paksoy, Pork & Shrimp"
1,Sopas,Soups,,,Sopa Vegetariana,Vegetarian Soup,"3,95",,1,1,"Paksoy, Bambu, Cogumelo e Verduras","Paksoy, Bamboo, Mushroom & Vegetables"
2,Arroz & Verduras,Rice & Vegetables,,,Arroz branco,Steamed Rice,"1,95",,1,1,Arroz ao vapor,Steamed Rice
3,Arroz & Verduras,Rice & Vegetables,,,Paksoy ao vapor,Steamed Paksoy,"2,40",,1,1,Paksoy ao vapor com Coentros,Steamed Paksoy with Coriander
4,Siao Long Pao,Siao Long Pao,,,Siao Long Pao Tradicional,Traditional Siao Long Pao,"4,50",,3,1,"Porco, Paksoy e Cogumelo Shiitake","Pork, Paksoy & Shiitake Mushroom"
